# Form 1

In [13]:
import pandas as pd
import logging
import datetime
from indices import form_1_indices as INDICES


csv_list = ['./Reports/1.csv', './Reports/2.csv', './Reports/3.csv', './Reports/4.csv']
# Add Logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
fh = logging.FileHandler('transform_logs.log', encoding='utf-8')
fh.setLevel(logging.DEBUG)
logger.addHandler(fh)
# Add name
NAME = 'report_form_1.py'

def construct_df(csv_list):
    '''
    Linear time:
    ~12 sec for 1 day
    ~6 min for 1 month
    '''
    def create_pivot(df):
        # Run Transforms for this day
        pivot_df_mistakes = df.pivot_table(index='Имя колл-листа', columns='Дата', values='Ошибки', aggfunc='sum')
        pivot_df_mistakes = pivot_df_mistakes.fillna(0)
        pivot_df_mistakes = pivot_df_mistakes.replace(0.00, '')
        pivot_df_mistakes.columns = pd.to_datetime(pivot_df_mistakes.columns, format='%d.%m.%Y')  # Fix date Time
        pivot_df_mistakes = pivot_df_mistakes.sort_index(axis=1)  # Fix date Time
        tmp_pivot_df_mistakes = pivot_df_mistakes.copy()  # Fix %%
        pivot_df_mistakes.index = pivot_df_mistakes.index + ' (ошибки шт.)'
        # Create dynamic Calls count (2)
        pivot_df_calls = df.pivot_table(index='Имя колл-листа', columns='Дата', values='Результат автооценки', aggfunc='count', fill_value=0)
        pivot_df_calls.columns = pd.to_datetime(pivot_df_calls.columns, format='%d.%m.%Y')  # Fix date Time
        pivot_df_calls = pivot_df_calls.sort_index(axis=1)  # Fix date Time
        tmp_pivot_df_calls = pivot_df_calls.copy()  # Fix %%
        pivot_df_calls.index = pivot_df_calls.index + ' (всего шт.)'
        # Create dynamic Mean Autoscore (3)
        pivot_df_mean = df.pivot_table(index='Имя колл-листа', columns='Дата', values='Результат автооценки', aggfunc='mean', fill_value='')
        pivot_df_mean.columns = pd.to_datetime(pivot_df_mean.columns, format='%d.%m.%Y')  # Fix date Time
        pivot_df_mean = pivot_df_mean.sort_index(axis=1)  # Fix date Time
        pivot_df_mean.index = pivot_df_mean.index + ' (средняя АО)'
        # Create dynamic Error Percentage (4)
        pivot_df_mistakes_filled = tmp_pivot_df_mistakes.replace('', 0)
        pivot_df_error_rate = (pivot_df_mistakes_filled / tmp_pivot_df_calls).applymap(lambda x: x if not pd.isna(x) else '')
        pivot_df_error_rate.index = pivot_df_error_rate.index + ' (доля ошибок %)'
        # Create Mega-Pivot
        # Concatenate the pivot tables vertically along rows (axis=0)
        pivot_table = pd.concat([pivot_df_mistakes, pivot_df_calls, pivot_df_mean, pivot_df_error_rate], axis=0)
        pivot_table = pivot_table.sort_index()
        return pivot_table
    # Concatenate all csv to a single biiig df
    df_main = pd.DataFrame(index=INDICES)
    df_rpc = pd.DataFrame(index=INDICES)
    for i in csv_list:
        '''
        Take report files 1-by-1 and the merge then on external index from indices.py
        This will cut RAM cost 30 times (and make shit slower)
        '''
        # Merge 2 frames
        df = pd.read_csv(i, sep=';', encoding='utf-8',header=0)
        # Remove мультидоговоры for RSB
        mask = df['№ п/п'].isna()
        df = df[~mask]
        # Convert the 'Длительность звонка' column to Timedelta
        df['Длительность звонка'] = pd.to_timedelta(df['Длительность звонка'])
        df['Ошибки'] = df['Результат автооценки'] != 100
        # Fix Date
        df['Дата'] = pd.to_datetime(df['Дата звонка'], format='%d.%m.%Y %H:%M:%S')
        df['Дата'] = df['Дата'].dt.strftime('%d.%m.%Y')
        df = df.reset_index(drop=True)
        # Create RPC
        rpc_df = df[df['Контактное лицо'] == 'Должник']
        rpc_df = rpc_df.reset_index(drop=True)
        # Warn if dates != 1
        if len(df['Дата'].unique().tolist()) > 1:
            logger.warning('%s Warning: more than a single date in df...', datetime.datetime.now())

        # MEMORY MANAGEMENT: CONCAT TO INDEX AND DELETE
        main_pivot = create_pivot(df)
        df_main = pd.concat([df_main, main_pivot], axis=1)
        del main_pivot  # Save 10MB
        rpc_pivot = create_pivot(rpc_df)
        df_rpc = pd.concat([df_rpc, rpc_pivot], axis=1)
        del rpc_pivot  # Save 10MB
    # Returns 2 complete pivots
    return df_main, df_rpc

def construct_summary(df_main, df_rpc):
    '''
    Calculations for the summary sheet
    WEIGHTED BT DAY -> Simple
    '''
    def create_col(pivot, title):
        calls = pivot[pivot.index.str.contains('(всего шт.)')].apply(pd.to_numeric, errors='coerce')
        calls = calls.sum(axis=1)
        error_rate = pivot[pivot.index.str.contains('(доля ошибок %)')].apply(pd.to_numeric, errors='coerce')
        error_rate = error_rate.mean(axis=1,skipna=True,numeric_only=True)
        errors = pivot[pivot.index.str.contains('(ошибки шт.)')].apply(pd.to_numeric, errors='coerce')
        errors = errors.sum(axis=1)
        score = pivot[pivot.index.str.contains('(средняя АО)')].apply(pd.to_numeric, errors='coerce')
        score = score.mean(axis=1,skipna=True, numeric_only=True)
        summary = pd.DataFrame(index=INDICES)
        summary[f'Свод: {title}'] = pd.concat([calls, error_rate, errors, score], axis=0)
        summary = summary.sort_index()
        return summary
    # Create Summary DF
    df_summary = pd.DataFrame()
    df_summary = pd.concat([create_col(df_main, 'все звонки'),
                            create_col(df_rpc, 'RPC') ], axis=1)
    # Returns Dataframe
    return df_summary

# Form 2

In [200]:
import pandas as pd
import logging
import datetime
from indices import form_2_indices as INDICES


csv_list = ['15GB.csv']
# Add Logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
fh = logging.FileHandler('transform_logs.log', encoding='utf-8')
fh.setLevel(logging.DEBUG)
logger.addHandler(fh)
# Add name
NAME = 'report_form_2.py'

In [201]:
def construct_df(csv_list):
    '''
    Linear time:
    ~9 sec for 1 day
    ~5 min for 1 month
    '''
    def create_pivot(df):
        # Create multiindex
        def create_multiindex(dataframe, sub_index:str):
            # Create MultiIndex
            multiindex = []
            for i, column in enumerate(dataframe):
                multiindex.append((column, sub_index))
            dataframe.columns = pd.MultiIndex.from_tuples(multiindex)
            return dataframe
        # Create Pivot FUNC
        pivot_df_calls = df.pivot_table(index=['Имя колл-листа', 'Результат робота'], columns='Дата', values='Результат автооценки', aggfunc='count', fill_value='')
        pivot_df_calls.columns = pd.to_datetime(pivot_df_calls.columns, format='%d.%m.%Y')  # Fix date Time
        pivot_df_calls = pivot_df_calls.sort_index(axis=1)  # Fix date Time
        # Calculate number of errors
        pivot_df_errors = df.pivot_table(index=['Имя колл-листа', 'Результат робота'], columns='Дата', values='Ошибки', aggfunc='sum', fill_value='')
        pivot_df_errors.columns = pd.to_datetime(pivot_df_errors.columns, format='%d.%m.%Y')  # Fix date Time
        pivot_df_errors = pivot_df_errors.sort_index(axis=1)  # Fix date Time
        # Calculate mean autoscore
        pivot_df_mean = df.pivot_table(index=['Имя колл-листа', 'Результат робота'], columns='Дата', values='Результат автооценки', aggfunc='mean', fill_value='')
        pivot_df_mean.columns = pd.to_datetime(pivot_df_mean.columns, format='%d.%m.%Y')  # Fix date Time
        pivot_df_mean = pivot_df_mean.sort_index(axis=1)  # Fix date Time
        # Calculate error rate
        pivot_df_error_rate = (pivot_df_errors.replace("", pd.NA) / pivot_df_calls.replace("", pd.NA)).applymap(lambda x: x if not pd.isna(x) else '')
        pivot_df_error_rate.columns = pd.to_datetime(pivot_df_error_rate.columns, format='%d.%m.%Y')  # Fix date Time
        pivot_df_error_rate = pivot_df_error_rate.sort_index(axis=1)  # Fix date Time
        # Create MultiIndex
        pivot_df_calls = create_multiindex(pivot_df_calls, 'Зв.(шт.)')
        pivot_df_errors = create_multiindex(pivot_df_errors, 'Ошб.(шт.)')
        pivot_df_mean = create_multiindex(pivot_df_mean, 'Ср.АО')
        pivot_df_error_rate = create_multiindex(pivot_df_error_rate, 'Ошб.%')
        # Create a list of the DataFrames you want to merge
        #dfs_to_merge = [pivot_df_calls, pivot_df_errors, pivot_df_mean, pivot_df_error_rate]
        dfs_to_merge = [pivot_df_error_rate, pivot_df_errors, pivot_df_calls, pivot_df_mean]
        # Initialize an empty DataFrame with the same index as the original DataFrames
        merged_df = pd.DataFrame(index=pivot_df_calls.index)
        # Create Multiindex
        multi_index = []
        # Iterate through the DataFrames and concatenate their columns in the desired order
        for num, column in enumerate(pivot_df_calls.columns):
                for dataframe in dfs_to_merge:
                        col_name = (dataframe.iloc[:, num].name[0], dataframe.iloc[:, num].name[1])
                        # Append the column name tuple to the list
                        multi_index.append(col_name)
                        merged_df[col_name] = dataframe.iloc[:, num]
        merged_df.columns = pd.MultiIndex.from_tuples(multi_index)
        # Returns merged df
        return merged_df
    # Create Base dfs for pivots
    multi_index = pd.MultiIndex.from_tuples(INDICES)
    multi_header = pd.MultiIndex.from_tuples([('tmp1','tmp2')])
    # Create your empty DataFrames with the MultiIndex
    df_main = pd.DataFrame(index=multi_index, columns=multi_header)
    df_rpc = pd.DataFrame(index=multi_index, columns=multi_header)
    for i in csv_list:
        '''
        Take report files 1-by-1 and the merge then on external index from indices.py
        This will cut RAM cost 30 times (and make shit slower)
        '''
        # Merge 2 frames
        df = pd.read_csv(i, sep=';', encoding='utf-8',header=0)
        # Remove мультидоговоры for RSB
        mask = df['№ п/п'].isna()
        df = df[~mask]
        # Convert the 'Длительность звонка' column to Timedelta
        df['Длительность звонка'] = pd.to_timedelta(df['Длительность звонка'])
        df['Ошибки'] = df['Результат автооценки'] != 100
        # Fix Date
        df['Дата'] = pd.to_datetime(df['Дата звонка'], format='%d.%m.%Y %H:%M:%S')
        df['Дата'] = df['Дата'].dt.strftime('%d.%m.%Y')
        df = df.reset_index(drop=True)
        # Create RPC
        rpc_df = df[df['Контактное лицо'] == 'Должник']
        rpc_df = rpc_df.reset_index(drop=True)
        # Warn if dates != 1
        if len(df['Дата'].unique().tolist()) > 1:
            logger.warning('%s Warning: more than a single date in df...', datetime.datetime.now())
        # MEMORY MANAGEMENT: CONCAT TO INDEX AND DELETE
        main_pivot = create_pivot(df)
        df_main = pd.concat([df_main, main_pivot], axis=1)
        del main_pivot  # Save 10MB
        rpc_pivot = create_pivot(rpc_df)
        df_rpc = pd.concat([df_rpc, rpc_pivot], axis=1)
        del rpc_pivot  # Save 10MB
    # Remove TMP columns
    del df_main[('tmp1','tmp2')]
    del df_rpc[('tmp1','tmp2')]
    # Returns 2 complete pivots
    return df_main, df_rpc

def construct_summary(df_main, df_rpc):
    '''
    Calculations for the summary sheet
    WEIGHTED BT DAY -> Simple
    '''
    def create_col(pivot, title):
        df = pivot
        errors_percent = df.loc[:, df.columns.get_level_values(1) == 'Ошб.%']
        errors_percent = errors_percent.mean(axis=1, skipna=True)
        errors_count = df.loc[:, df.columns.get_level_values(1) == 'Ошб.(шт.)']
        errors_count = errors_count.sum(axis=1, numeric_only=True)
        calls_count = df.loc[:, df.columns.get_level_values(1) == 'Зв.(шт.)']
        calls_count = calls_count.sum(axis=1, numeric_only=True)
        score = df.loc[:, df.columns.get_level_values(1) == 'Ср.АО']
        score = score.mean(axis=1, skipna=True)
        df = pd.DataFrame(index=pd.MultiIndex.from_tuples(df.index), columns=pd.MultiIndex.from_tuples([(title,'')]))
        # Mask Error Count
        mask = calls_count == 0
        df[(title, 'Ошб.%')] = errors_percent
        df[(title, 'Ошб.(шт.)')] = errors_count[~mask]  # pd.mean considers NA = 0
        df[(title, 'Зв.(шт.)')] = calls_count.replace(0,pd.NA)
        df[(title, 'Ср.АО')] = score
        del df[(title, '')]
        # Returns weighted sumary
        return df
        # Create Summary DF
    df_summary = pd.DataFrame()
    df_summary = pd.concat([create_col(df_main, 'Срез: все звонки'),
                            create_col(df_rpc, 'Срез: RPC') ], axis=1)
    # Returns Dataframe
    return df_summary

# Form 3 Big

In [9]:
import pandas as pd
import logging
import datetime
from indices import form_1_indices as INDICES


#csv_list = ['./Reports/1.csv', './Reports/2.csv', './Reports/3.csv', './Reports/4.csv']
csv_list = ['./15GB.csv']
# Add Logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
fh = logging.FileHandler('transform_logs.log', encoding='utf-8')
fh.setLevel(logging.DEBUG)
logger.addHandler(fh)
# Add name
NAME = 'report_form_3.py'

In [10]:
def construct_df(csv_list):
    '''
    Linear time:
    ~9 sec for 1 day
    ~5 min for 1 month
    '''
    def create_pivot(df):
         # GET HEADERS AND REMOVE IRRELEVANT ONES
        queries_list = list(df.columns)
        columns_to_remove = [
            '№ п/п',
            'ID звонка',
            'Имя колл-листа',
            'Результат робота',
            'Дата звонка',
            'Результат автооценки',
            'Ошибки',
            'Дата',
            'Поисковый запрос: Все звонки, балл',
            'Контактное лицо',
            'Длительность звонка',
            'Тип договора'
        ]
        # Remove the specified columns
        queries_list = [col for col in queries_list if col not in columns_to_remove]
        # Все звонки fix: для отображния стрроки 'Все звонки'
        df['Всего звонков по листу'] = df['Поисковый запрос: Все звонки, балл']
        queries_list.append('Всего звонков по листу')
        # Each column has -25/0, by dividing I get a 1/0 format
        for col in queries_list:
            df[col] = df[col].astype('Int8')  # Limit RAM usage
            df[col] = df[col] / df[col]
        queries_list.append('Ошибки')
        # INTENSIVE MELTING
        # First, melt the DataFrame to convert 'Запрос_1', 'Запрос_2', 'Запрос_3' into rows
        melted_df = pd.melt(df, id_vars=['Имя колл-листа', 'Дата'], value_vars = queries_list, var_name='Запрос', value_name='Ошибки шт.')
        melted_df.reset_index(drop=True)
        #del df
        # Now, create a pivot table to calculate sums
        pivot_table = melted_df.pivot_table(
            values='Ошибки шт.',
            index=['Имя колл-листа', 'Запрос'],
            columns='Дата',
            aggfunc='sum'
        )
        # del melted_df
        pivot_table = pivot_table.reset_index()
        return pivot_table
    
    for i in csv_list:
            '''
            Take report files 1-by-1 and the merge then on external index from indices.py
            This will cut RAM cost 30 times (and make shit slower)
            '''
            # Merge 2 frames
            df = pd.read_csv(i, sep=';', encoding='utf-8',header=0)
            # Remove мультидоговоры for RSB
            mask = df['№ п/п'].isna()
            df = df[~mask]
            # Convert the 'Длительность звонка' column to Timedelta
            df['Длительность звонка'] = pd.to_timedelta(df['Длительность звонка'])
            df['Ошибки'] = df['Результат автооценки'] != 100
            # Fix Date
            df['Дата'] = pd.to_datetime(df['Дата звонка'], format='%d.%m.%Y %H:%M:%S')
            df['Дата'] = df['Дата'].dt.strftime('%d.%m.%Y')
            df = df.reset_index(drop=True)
            # Create RPC
            rpc_df = df[df['Контактное лицо'] == 'Должник']
            rpc_df = rpc_df.reset_index(drop=True)
            # Warn if dates != 1
            if len(df['Дата'].unique().tolist()) > 1:
                logger.warning('%s Warning: more than a single date in df...', datetime.datetime.now())

            # MEMORY MANAGEMENT: CONCAT TO INDEX AND DELETE
            main_pivot = create_pivot(df)
            rpc_pivot = create_pivot(rpc_df)
            return main_pivot, rpc_pivot

In [17]:
# GET HEADERS AND REMOVE IRRELEVANT ONES
queries_list = list(df.columns)
columns_to_remove = [
    '№ п/п',
    'ID звонка',
    'Имя колл-листа',
    'Результат робота',
    'Дата звонка',
    'Результат автооценки',
    'Ошибки',
    'Дата',
    'Поисковый запрос: Все звонки, балл',
    'Контактное лицо',
    'Длительность звонка',
    'Тип договора'
]
# Remove the specified columns
queries_list = [col for col in queries_list if col not in columns_to_remove]
# Все звонки fix: для отображния стрроки 'Все звонки'
df['Всего звонков по листу'] = df['Поисковый запрос: Все звонки, балл']
queries_list.append('Всего звонков по листу')
# Each column has -25/0, by dividing I get a 1/0 format
for col in queries_list:
    df[col] = df[col].astype('Int8')  # Limit RAM usage
    df[col] = df[col] / df[col]
queries_list.append('Ошибки')
# INTENSIVE MELTING
# First, melt the DataFrame to convert 'Запрос_1', 'Запрос_2', 'Запрос_3' into rows
melted_df = pd.melt(df, id_vars=['Имя колл-листа', 'Дата'], value_vars = queries_list, var_name='Запрос', value_name='Ошибки шт.')
melted_df.reset_index(drop=True)
#del df
# Now, create a pivot table to calculate sums
pivot_table = melted_df.pivot_table(
    values='Ошибки шт.',
    index=['Имя колл-листа', 'Запрос'],
    columns='Дата',
    aggfunc='sum'
)
# del melted_df
pivot_table = pivot_table.reset_index()

In [55]:
excel_file_path = 'form3.xlsx'
sheet_name = 'TEST'

# Define a green data bar format
green_databar_format = {
    'type': 'data_bar',
    'bar_color': '#63C384',  # Hex color code for green
}

# Define a red data bar format
red_databar_format = {
    'type': 'data_bar',
    'bar_color': '#e85f5f',  # Hex color code for green
}
color_scale_rule_percent = {
                'type': '2_color_scale',
                'min_color': '#FFFFFF',  # White
                'max_color': '#e85f5f',  # Red
                'min_type': 'num',
                'min_value': 0,
                'max_type': 'percentile',
                'max_value': 100
                }

with pd.ExcelWriter(excel_file_path, engine='xlsxwriter') as writer:
    # Write the DataFrame to the Excel file
    pivot_table.to_excel(writer, sheet_name=sheet_name)
    # Access the xlsxwriter workbook and worksheet objects
    workbook = writer.book
    worksheet = writer.sheets[sheet_name]
    white_fill_format = workbook.add_format({'text_wrap': False, 'bg_color': '#FFFFFF', 'border': 0})
    ref_format = workbook.add_format({'text_wrap': False, 'bold': True, 'align':'right', 'bg_color': '#FFFFFF', 'border': 0})
    worksheet.set_column(0, 0, 5, white_fill_format)
    worksheet.set_column(1, 1, 25, white_fill_format)
    worksheet.set_column(2, 2, 100, white_fill_format)
    worksheet.set_column(3, 100, 10, white_fill_format)
    format_tmp_start = []
    format_tmp_stop = []
    for row_num, row in enumerate(pivot_table['Запрос']):
        if "Всего звонков по листу" in row:
            worksheet.conditional_format(f'D{row_num+2}:AZ{row_num+2}', green_databar_format)
            worksheet.write(f'C{row_num+2}', 'Всего звонков по листу',ref_format)
            format_tmp_start.append(row_num+4)
            format_tmp_stop.append(row_num+1)  # Hack I append a list of previous querie
        elif "Ошибки" in row:
            worksheet.conditional_format(f'D{row_num+2}:AZ{row_num+2}', red_databar_format)
            worksheet.write(f'C{row_num+2}', 'Ошибки', ref_format)
    # Prepare ranges for formatting
    format_tmp_start = format_tmp_start[:-1]
    format_tmp_stop = format_tmp_stop[1:]
    for num, i in enumerate(format_tmp_start):
        worksheet.conditional_format(f'D{i}:AZ{format_tmp_stop[num]}', color_scale_rule_percent)
        


In [40]:
print(format_tmp_start[:-1])
print(format_tmp_stop[1:])

[4, 54, 104, 154, 204, 254, 304, 354, 404, 454, 504, 554, 604, 654, 704, 754, 804, 854, 904, 954, 1004, 1054, 1104, 1154, 1204, 1254, 1304]
[101, 151, 201, 251, 301, 351, 401, 451, 501, 551, 601, 651, 701, 751, 801, 851, 901, 951, 1001, 1051, 1101, 1151, 1201, 1251, 1301, 1351, 1401]


In [15]:
call_list = ['Имя колл-листа',
'Актуализация',
'Внебаланс (new)',
'ДИЗ',
'ДИЗ (возражения)',
'ДИЗ (чередование)',
'Ипотека',
'Исполнительное производство',
'Исполнительное производство (new)',
'Коллекшн ранний сбор',
'Матрица мотиваторов',
'Матрица мотиваторов (распознавание эмоций new)',
'Матрица мотиваторов (распознавание эмоций)',
'Матрица мотиваторов Хард. Ранний сбор',
'Матрица мотиваторов Хард. Ранний сбор (кредитные карты)',
'Матрица мотивации. Выезд',
'Мультидоговоры. Универсальный',
'Неконтактные',
'Неконтактные (внебаланс)',
'Неконтактные (исполка)',
'Неконтактные (поздний сбор)',
'Неконтактные Суд',
'Ожидание выезда',
'Ожидание суда',
'Подготовка к передаче в КА',
'Преколлекшн (rename)',
'Суд',
'Усиление ПС Предцессия',
'Усиление позднего сбора',
'ФЗ-230']

queries = [
    'Запрос',
'Всего звонков по листу',
'Ошибки',
'Поисковый запрос: Выслушайте меня, балл',
'Поисковый запрос: Выявление тишины, балл',
'Поисковый запрос: Диалог с автоответчиком &gt; 30 сек. без переводов, балл',
'Поисковый запрос: Длительные диалоги "Бросил трубку", балл',
'Поисковый запрос: Для автооценки: мобилизация робот не понимает, балл',
'Поисковый запрос: Звонки не с 900, балл',
'Поисковый запрос: Клиент подал на рестру, робот не диктует номер, балл',
'Поисковый запрос: Не отвечает на вопрос про сумму, балл',
'Поисковый запрос: Не отрабатывает возражение "Банкротство", балл',
'Поисковый запрос: Не отрабатывает возражение "Декрет", балл',
'Поисковый запрос: Не отрабатывает возражение "Задержка з/п", балл',
'Поисковый запрос: Не отрабатывает возражение "Клиент обратился к антиколлекторам", балл',
'Поисковый запрос: Не отрабатывает возражение "Не согласен с суммой", балл',
'Поисковый запрос: Не отрабатывает возражение "Нет денег", балл',
'Поисковый запрос: Не отрабатывает возражение "Нет работы", балл',
'Поисковый запрос: Не отрабатывает возражение "Оплата позже", балл',
'Поисковый запрос: Не отрабатывает возражение "Оплата частями", балл',
'Поисковый запрос: Не отрабатывает возражение "Перекладывание ответственности", балл',
'Поисковый запрос: Не отрабатывает возражение "Ситуация в стране/регионе", балл',
'Поисковый запрос: Не отрабатывает возражение "Сложные жизненные обстоятельства", балл',
'Поисковый запрос: Не отрабатывает возражение "Угрозы", балл',
'Поисковый запрос: Не отрабатывает возражение "Уже работаю с приставами", балл',
'Поисковый запрос: Не отрабатывает возражение "Я болею", балл',
'Поисковый запрос: Не отрабатывает возражение «Подавайте в суд», балл',
'Поисковый запрос: Не отрабатывает возражение «Эмоциональные возражения», балл',
'Поисковый запрос: Не переводит на оператора клиента с группой инвалидности, балл',
'Поисковый запрос: Неправильная активность в АС Калита (вместо "Контакт с родственником" фиксирует "Контакт с третьим лицом"), балл',
'Поисковый запрос: Неправомерное обещание (клиент не подтверждает оплату), балл',
'Поисковый запрос: Неправомерный отказ 1, балл',
'Поисковый запрос: Обещание/отказ в Калите вместо "Нужна помощь оператора", балл',
'Поисковый запрос: Отработка лжи Общ, балл',
'Поисковый запрос: Ошибка идентификации: не уточняет дату рождения, балл',
'Поисковый запрос: Перебивает и не даёт сказать дату до конца, балл',
'Поисковый запрос: При переводе на оператора Робот молчит более 10 сек, балл',
'Поисковый запрос: Разглашение БТ, балл',
'Поисковый запрос: Разговор с эхом &gt; 1 мин, балл',
'Поисковый запрос: Робот не ответил на "а вдруг вы мошенники", балл',
'Поисковый запрос: Робот не отвечает на вопрос "какой банк", балл',
'Поисковый запрос: Робот не отвечает на вопрос про дату, балл',
'Поисковый запрос: Робот не переходит по скрипту при просьбе клиента перезвонить позже, балл',
'Поисковый запрос: Робот не переходит по скрипту, когда клиент уточняет причину звонка (Pre-Collection), балл',
'Поисковый запрос: Робот не предлагает сервис трудоустройства, балл',
'Поисковый запрос: Робот некорректно фиксирует "Перезвонить", балл',
'Поисковый запрос: Робот некорректно фиксирует Заявление о факте платежа - Исполнительное производство + Внебаланс, балл',
'Поисковый запрос: Робот некорректно фиксирует Заявление о факте платежа, балл',
'Поисковый запрос: Спрашивает дату, когда не нужно, балл',
'Поисковый запрос: вместо тел не принадлежит - отложить звонок, балл',
'Поисковый запрос: для автооценки: Параллельный диалог с клиентом, балл'
]
mega_index = []
for cl in queries_list:
    multiindex_3 =[
    (cl, 'Блок: 1. Идентификация', 'Поисковый запрос: Ошибка идентификации: не уточняет дату рождения, балл'),
    (cl, 'Блок: 1. Идентификация', 'Поисковый запрос: Спрашивает дату, когда не нужно, балл'),
    (cl, 'Блок: 1. Идентификация', 'Поисковый запрос: Перебивает и не даёт сказать дату до конца, балл'),
    (cl, 'Блок: 1. Идентификация', 'Поисковый запрос: Разглашение БТ, балл'),
    (cl, 'Блок: 1. Идентификация', 'Поисковый запрос: Диалог с автоответчиком > 30 сек. без переводов, балл'),
    (cl, 'Блок: 1. Идентификация', 'Поисковый запрос: Длительные диалоги "Бросил трубку", балл'),
    (cl, 'Блок: 2. Мотивация', 'Поисковый запрос: При переводе на оператора Робот молчит более 10 сек, балл'),
    (cl, 'Блок: 2. Мотивация', 'Поисковый запрос: Выслушайте меня, балл'),
    (cl, 'Блок: 2. Мотивация', 'Поисковый запрос: Разговор с эхом > 1 мин, балл'),
    (cl, 'Блок: 2. Мотивация', 'Поисковый запрос: Робот не предлагает сервис трудоустройства, балл'),
    (cl, 'Блок: 2. Мотивация', 'Поисковый запрос: для автооценки: Параллельный диалог с клиентом, балл'),
    (cl, 'Блок: 2. Мотивация', 'Поисковый запрос: Клиент подал на рестру, робот не диктует номер, балл'),
    (cl, 'Блок: 2. Мотивация', 'Поисковый запрос: Для автооценки: мобилизация робот не понимает, балл'),
    (cl, 'Блок: 2. Мотивация', 'Поисковый запрос: Выявление тишины, балл'),
    (cl, 'Блок: 2. Мотивация', 'Поисковый запрос: При переводе на оператора Робот молчит более 10 сек, балл'),
    (cl, 'Блок: 3. Отработка возражений', 'Поисковый запрос: Робот не отвечает на вопрос про дату, балл'),
    (cl, 'Блок: 3. Отработка возражений', 'Поисковый запрос: Не отрабатывает возражение "Банкротство", балл'),
    (cl, 'Блок: 3. Отработка возражений', 'Поисковый запрос: Робот не ответил на "а вдруг вы мошенники", балл'),
    (cl, 'Блок: 3. Отработка возражений', 'Поисковый запрос: Робот не отвечает на вопрос "какой банк", балл'),
    (cl, 'Блок: 3. Отработка возражений', 'Поисковый запрос: Звонки не с 900, балл'),
    (cl, 'Блок: 3. Отработка возражений', 'Поисковый запрос: Не отвечает на вопрос про сумму, балл'),
    (cl, 'Блок: 3. Отработка возражений', 'Поисковый запрос: Робот не переходит по скрипту, когда клиент уточняет причину звонка (Pre-Collection), балл'),
    (cl, 'Блок: 3. Отработка возражений', 'Поисковый запрос: Не отрабатывает возражение "Оплата позже", балл'),
    (cl, 'Блок: 3. Отработка возражений', 'Поисковый запрос: Не отрабатывает возражение "Задержка з/п", балл'),
    (cl, 'Блок: 3. Отработка возражений', 'Поисковый запрос: Не отрабатывает возражение "Нет денег", балл'),
    (cl, 'Блок: 3. Отработка возражений', 'Поисковый запрос: Не отрабатывает возражение "Нет работы", балл'),
    (cl, 'Блок: 3. Отработка возражений', 'Поисковый запрос: Не отрабатывает возражение "Не согласен с суммой", балл'),
    (cl, 'Блок: 3. Отработка возражений', 'Поисковый запрос: Не отрабатывает возражение "Оплата частями", балл'),
    (cl, 'Блок: 3. Отработка возражений', 'Поисковый запрос: Не отрабатывает возражение «Подавайте в суд», балл'),
    (cl, 'Блок: 3. Отработка возражений', 'Поисковый запрос: Не отрабатывает возражение «Эмоциональные возражения», балл'),
    (cl, 'Блок: 3. Отработка возражений', 'Поисковый запрос: Не отрабатывает возражение "Ситуация в стране/регионе", балл'),
    (cl, 'Блок: 3. Отработка возражений', 'Поисковый запрос: Не отрабатывает возражение "Угрозы", балл'),
    (cl, 'Блок: 3. Отработка возражений', 'Поисковый запрос: Не отрабатывает возражение "Я болею", балл'),
    (cl, 'Блок: 3. Отработка возражений', 'Поисковый запрос: Не отрабатывает возражение "Сложные жизненные обстоятельства", балл'),
    (cl, 'Блок: 3. Отработка возражений', 'Поисковый запрос: Не отрабатывает возражение "Клиент обратился к антиколлекторам", балл'),
    (cl, 'Блок: 3. Отработка возражений', 'Поисковый запрос: Не отрабатывает возражение "Перекладывание ответственности", балл'),
    (cl, 'Блок: 3. Отработка возражений', 'Поисковый запрос: Не отрабатывает возражение "Уже работаю с приставами", балл'),
    (cl, 'Блок: 3. Отработка возражений', 'Поисковый запрос: Не отрабатывает возражение "Декрет", балл'),
    (cl, 'Блок: 4. Фиксация обещаний об оплате', 'Поисковый запрос: Отработка лжи Общ, балл'),
    (cl, 'Блок: 4. Фиксация обещаний об оплате', 'Поисковый запрос: Робот некорректно фиксирует Заявление о факте платежа, балл'),
    (cl, 'Блок: 4. Фиксация обещаний об оплате', 'Поисковый запрос: Робот некорректно фиксирует Заявление о факте платежа - Исполнительное производство + Внебаланс, балл'),
    (cl, 'Блок: 4. Фиксация обещаний об оплате', 'Поисковый запрос: Неправомерное обещание (клиент не подтверждает оплату), балл'),
    (cl, 'Блок: 5. Фиксация обещаний об оплате', 'Поисковый запрос: вместо тел не принадлежит - отложить звонок, балл'),
    (cl, 'Блок: 5. Фиксация обещаний об оплате', 'Поисковый запрос: Неправомерный отказ 1, балл'),
    (cl, 'Блок: 5. Фиксация обещаний об оплате', 'Поисковый запрос: Неправильная активность в АС Калита (вместо "Контакт с родственником" фиксирует "Контакт с третьим лицом"), балл'),
    (cl, 'Блок: 5. Фиксация обещаний об оплате', 'Поисковый запрос: Не переводит на оператора клиента с группой инвалидности, балл'),
    (cl, 'Блок: 5. Фиксация обещаний об оплате', 'Поисковый запрос: Обещание/отказ в Калите вместо "Нужна помощь оператора", балл'),
    (cl, 'Блок: 5. Фиксация обещаний об оплате', 'Поисковый запрос: Робот не переходит по скрипту при просьбе клиента перезвонить позже, балл'),
    (cl, 'Блок: 5. Фиксация обещаний об оплате', 'Поисковый запрос: Робот некорректно фиксирует "Перезвонить", балл')
    ]
    mega_index.append(multiindex_3)

cl
multiindex_3 =[
(cl, 'Блок: 1. Идентификация', 'Поисковый запрос: Ошибка идентификации: не уточняет дату рождения, балл'),
(cl, 'Блок: 1. Идентификация', 'Поисковый запрос: Спрашивает дату, когда не нужно, балл'),
(cl, 'Блок: 1. Идентификация', 'Поисковый запрос: Перебивает и не даёт сказать дату до конца, балл'),
(cl, 'Блок: 1. Идентификация', 'Поисковый запрос: Разглашение БТ, балл'),
(cl, 'Блок: 1. Идентификация', 'Поисковый запрос: Диалог с автоответчиком > 30 сек. без переводов, балл'),
(cl, 'Блок: 1. Идентификация', 'Поисковый запрос: Длительные диалоги "Бросил трубку", балл'),
(cl, 'Блок: 2. Мотивация', 'Поисковый запрос: При переводе на оператора Робот молчит более 10 сек, балл'),
(cl, 'Блок: 2. Мотивация', 'Поисковый запрос: Выслушайте меня, балл'),
(cl, 'Блок: 2. Мотивация', 'Поисковый запрос: Разговор с эхом > 1 мин, балл'),
(cl, 'Блок: 2. Мотивация', 'Поисковый запрос: Робот не предлагает сервис трудоустройства, балл'),
(cl, 'Блок: 2. Мотивация', 'Поисковый запрос: для автооценки: Параллельный диалог с клиентом, балл'),
(cl, 'Блок: 2. Мотивация', 'Поисковый запрос: Клиент подал на рестру, робот не диктует номер, балл'),
(cl, 'Блок: 2. Мотивация', 'Поисковый запрос: Для автооценки: мобилизация робот не понимает, балл'),
(cl, 'Блок: 2. Мотивация', 'Поисковый запрос: Выявление тишины, балл'),
(cl, 'Блок: 2. Мотивация', 'Поисковый запрос: При переводе на оператора Робот молчит более 10 сек, балл'),
(cl, 'Блок: 3. Отработка возражений', 'Поисковый запрос: Робот не отвечает на вопрос про дату, балл'),
(cl, 'Блок: 3. Отработка возражений', 'Поисковый запрос: Не отрабатывает возражение "Банкротство", балл'),
(cl, 'Блок: 3. Отработка возражений', 'Поисковый запрос: Робот не ответил на "а вдруг вы мошенники", балл'),
(cl, 'Блок: 3. Отработка возражений', 'Поисковый запрос: Робот не отвечает на вопрос "какой банк", балл'),
(cl, 'Блок: 3. Отработка возражений', 'Поисковый запрос: Звонки не с 900, балл'),
(cl, 'Блок: 3. Отработка возражений', 'Поисковый запрос: Не отвечает на вопрос про сумму, балл'),
(cl, 'Блок: 3. Отработка возражений', 'Поисковый запрос: Робот не переходит по скрипту, когда клиент уточняет причину звонка (Pre-Collection), балл'),
(cl, 'Блок: 3. Отработка возражений', 'Поисковый запрос: Не отрабатывает возражение "Оплата позже", балл'),
(cl, 'Блок: 3. Отработка возражений', 'Поисковый запрос: Не отрабатывает возражение "Задержка з/п", балл'),
(cl, 'Блок: 3. Отработка возражений', 'Поисковый запрос: Не отрабатывает возражение "Нет денег", балл'),
(cl, 'Блок: 3. Отработка возражений', 'Поисковый запрос: Не отрабатывает возражение "Нет работы", балл'),
(cl, 'Блок: 3. Отработка возражений', 'Поисковый запрос: Не отрабатывает возражение "Не согласен с суммой", балл'),
(cl, 'Блок: 3. Отработка возражений', 'Поисковый запрос: Не отрабатывает возражение "Оплата частями", балл'),
(cl, 'Блок: 3. Отработка возражений', 'Поисковый запрос: Не отрабатывает возражение «Подавайте в суд», балл'),
(cl, 'Блок: 3. Отработка возражений', 'Поисковый запрос: Не отрабатывает возражение «Эмоциональные возражения», балл'),
(cl, 'Блок: 3. Отработка возражений', 'Поисковый запрос: Не отрабатывает возражение "Ситуация в стране/регионе", балл'),
(cl, 'Блок: 3. Отработка возражений', 'Поисковый запрос: Не отрабатывает возражение "Угрозы", балл'),
(cl, 'Блок: 3. Отработка возражений', 'Поисковый запрос: Не отрабатывает возражение "Я болею", балл'),
(cl, 'Блок: 3. Отработка возражений', 'Поисковый запрос: Не отрабатывает возражение "Сложные жизненные обстоятельства", балл'),
(cl, 'Блок: 3. Отработка возражений', 'Поисковый запрос: Не отрабатывает возражение "Клиент обратился к антиколлекторам", балл'),
(cl, 'Блок: 3. Отработка возражений', 'Поисковый запрос: Не отрабатывает возражение "Перекладывание ответственности", балл'),
(cl, 'Блок: 3. Отработка возражений', 'Поисковый запрос: Не отрабатывает возражение "Уже работаю с приставами", балл'),
(cl, 'Блок: 3. Отработка возражений', 'Поисковый запрос: Не отрабатывает возражение "Декрет", балл'),
(cl, 'Блок: 4. Фиксация обещаний об оплате', 'Поисковый запрос: Отработка лжи Общ, балл'),
(cl, 'Блок: 4. Фиксация обещаний об оплате', 'Поисковый запрос: Робот некорректно фиксирует Заявление о факте платежа, балл'),
(cl, 'Блок: 4. Фиксация обещаний об оплате', 'Поисковый запрос: Робот некорректно фиксирует Заявление о факте платежа - Исполнительное производство + Внебаланс, балл'),
(cl, 'Блок: 4. Фиксация обещаний об оплате', 'Поисковый запрос: Неправомерное обещание (клиент не подтверждает оплату), балл'),
(cl, 'Блок: 5. Фиксация обещаний об оплате', 'Поисковый запрос: вместо тел не принадлежит - отложить звонок, балл'),
(cl, 'Блок: 5. Фиксация обещаний об оплате', 'Поисковый запрос: Неправомерный отказ 1, балл'),
(cl, 'Блок: 5. Фиксация обещаний об оплате', 'Поисковый запрос: Неправильная активность в АС Калита (вместо "Контакт с родственником" фиксирует "Контакт с третьим лицом"), балл'),
(cl, 'Блок: 5. Фиксация обещаний об оплате', 'Поисковый запрос: Не переводит на оператора клиента с группой инвалидности, балл'),
(cl, 'Блок: 5. Фиксация обещаний об оплате', 'Поисковый запрос: Обещание/отказ в Калите вместо "Нужна помощь оператора", балл'),
(cl, 'Блок: 5. Фиксация обещаний об оплате', 'Поисковый запрос: Робот не переходит по скрипту при просьбе клиента перезвонить позже, балл'),
(cl, 'Блок: 5. Фиксация обещаний об оплате', 'Поисковый запрос: Робот некорректно фиксирует "Перезвонить", балл')
]